# A statistical test of eigenvector angles to evaluate the similarity of neural network simulations
### Or how the analytic description of random angles and eigenvalues can help to compare matrices

In [1]:
import numpy as np
import scipy as sc
from quantities import Hz, ms
import math

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, fixed

from elephant.spike_train_correlation import corrcoef
from elephant.conversion import BinnedSpikeTrain
from elephant.spike_train_generation import homogeneous_poisson_process as HPP

## Table of Contents
* [An angle measure to evaluate similiarity between matrices](#intro)
    * [The context](#intro-a)
    * [The problem](#intro-b)
    * [The idea](#intro-c)
    * [The outline](#intro-d)
* [On the behavior of angles](#angles)
    * [Angles in high dimensions](#angles-a)
    * [Simulating random vectors](#angles-b)
    * [Simulating random eigenvectors](#angles-b)
* [Angles as similarity measure](#smallness)
    * [Angle-smallness](#smallness-a)
    * [Distribution of eigenvalues](#smallness-b)
    * [Weighted angle-smallness](#smallness-c)
* [Building a statistical test](#test)
    * [Similarity score](#test-a)
    * [Null hypothesis](#test-b)
    * [P-value and interpretation](#test-c)
* [Application to neural activity data](#apply)
    * [Practical considerations](#apply-a)    
    * [Example data](#apply-b)
    * [Parameter scan](#apply-c)

<a id='intro'></a>
# An angle measure to evaluate similiarity between matrices

<a id='intro-a'></a>
## The context
There are many ways to compare activity data of neural networks, both for data obtained from biological experiments and from model simulations. This is great, because the whole concept of model validation is based on the rigorous comparison between experimental and simulated data. A fairly straight-forward way to compare network activity data is via node-wise (i.e neuron-wise) measures, such as firing rates, spiking regularity, or graph centrality. These neuron-wise measures are statistically independent and can be represented by sample distributions which can be compared by the [wide variety of statistical two-sample tests](https://en.wikipedia.org/wiki/Two-sample_hypothesis_testing). Popular tests are for example, the Student's t-test, the Kolmogorov-Smirnov test, or the Wilcoxon rank-sum test. Even though they are not formally a statistical tests, also comparative measures such as the Kullback-Leibler divergence and the effect size have to be mentioned.

<a id='intro-b'></a>
## The problem
However, not all measures to characterize network activity are statistically independent. Since the key feature of a network is its interconectiveness, argueably the most interesting characteristics are captured by pairwise or high-order measures like correlation, effective connectivity, or Granger causality. Such measures can as well be represented by simple sample distributions, but this would neglect the dependence between the individual values. For example, neuron A has a certain correlation with neuron B, and neuron B a correlation with neuron C, then the correlation which is possible between neuron A and C is dependent by their correlations with neuron B. So in order to not loose this dependence, the values need to be represented not as a sample distribution but as a matrix. Comparing matrices in a meaningful way is however much more tricky than comparing distributions, and few statistical tests are available.

<a id='intro-c'></a>
### The idea
The various two-sample tests typically compare certain attributes of the distributions, e.g. the mean, the variance, or the shape. For the case of matrices we can analogously consider the eigenvalues and eigenvectors as relevant attributes. The eigenvectors span the space in which the data in the matrix is represented most naturally, in the sense that the first eigenvector points along the direction of the most variance in the data, the second along the direction of the most variance within the orthogonal subspace, and so on; whereas the corresponding eigenvalues quantify the variance along these axes. Thus, for comparison of two matrices, we could measure the similarity between them by means of the similariy of the eigenvectors, especially the first ones, i.e., the ones with the largest eigenvalues. And how to evaluate the similarity between vectors? Of course, angles!

<a id='intro-d'></a>
### The outline
In the following we will build up a theoretical basis to evaluate the similarity between matrices by considering the angles between eigenvectors.
First, we will describe the behavior of angles for high dimensional vectors, especially eigenvectors.
Then, we will look the smallness of angles as a indicator for similarity and integrate the eigenvalues as a weighting factor into the analytical description.
Based on this we formulate a similarity score for a matix comparison and describe its theoretical distribution to compute a corresponding p-value.
Finally, we apply this statistical test to compare neural network activity. 

<a id='angles'></a>
# The distribution of random angles

<a id='angles-a'></a>
## Angles in high dimensions
The basic idea is that a small angle between two vectors indicates similarity, a large angle indicates discrepancy, where the angle is defined as $\phi=\arccos(\mathbf{v}_1 \cdot \mathbf{v}_2)$. But in order to know when an angle is large and when it is small we need to relate it to a reference scenario, i.e. the null hypothesis. Here, the null hypothesis would be that both vectors point into a random direction, or to be precise they are random vectors uniformly distributed over the N-dim unit sphere. As the vectors exist in a 'neuron space' corresponding to the dimensionality of our matrix, N is equal to the number of neurons we record from.

This may seem overly formal since we just want to distinguish small and large angles, but this is crucial because angles behave very differently depending on the vector dimensions. In two dimension the angle distribution is still excatly how we would expect them to be. Each angle between random vectors is equally likely and thus the distribution is uniform between $0$ and $\pi$. However, once we leave the simple 2D world and move to higher dimensions things get fairly unintuitive. For example, consider ball of diameter d within a cube with edge length d. In 2 or 3 dimensions nearly all the volume of the cube is also within the ball, but in higher dimensions the volume concentrates more and more at the surface of the cube, so that it is outside of the ball. So that means, if you stand in a random spot in a high-dimensional classroom, you'll nearly always stand in a corner. Part of the same phenomenon, and relevant for our angles, is that in high dimensions random vector pairs are nearly always perpendicular.

Fortunately, the probability distribution $f_\sphericalangle(\phi)$ of random angles depending on the dimension $N$ can be described analytically as [(*Cai et al., 2014*)]()
$$
f_\sphericalangle(\phi) =  \frac{\Gamma(\frac{N}{2})}{\sqrt{\pi} \Gamma(\frac{N-1}{2})} \sin^{N-2}(\phi) \qquad \phi \in [0,\pi],
$$
or as written in Python:

In [2]:
def angle_dist(phi, N):
    if phi >= 0 and phi <= np.pi:
        return math.gamma(N/2.) / (np.sqrt(np.pi) \
             * math.gamma((N-1)/2)) * np.sin(phi)**(N-2)
    else:
        return 0

<a id='angles-b'></a>
## Angle distribution for random vectors

Let's see how this looks like in practice. First, we need to generate some uniformly distributed random vectors. This is most easily done by drawing each vector component from a normal distribution and then normalizing the vectors [(*Guhr et al., 1998*)]().

In [3]:
def generate_random_vectors(N, samples):
    random_vectors = np.random.normal(size=(samples, N))
    for i, v in enumerate(random_vectors):
        random_vectors[i] /= np.linalg.norm(v)
    return random_vectors

To visually check that this method actually produces uniformly distributed random vectors, we plot a bunch of them in a 3D plot.

In [4]:
%matplotlib widget

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')

rand_vectors = generate_random_vectors(N=3, samples=1000)

ax.scatter(rand_vectors[:,0],rand_vectors[:,1],rand_vectors[:,2], 
           color='r', s=20, label='random vectors');

FigureCanvasNbAgg()

The random vectors appear to be indeed uniformly distributed. 
The random angles we compute by sampling two sets of random vectors and instead of calculating the arcus cosine of the scalar product of each pair separately, we apply the equivalent operation of multiplying the two sets of vectors and calculating the arcus cosine on the diagonal of the resulting matrix.

In [5]:
def generate_random_angles(N, samples, vector_function):
    vectors_a = vector_function(N=N, samples=samples)
    vectors_b = vector_function(N=N, samples=samples)    
    M = np.dot(vectors_a, vectors_b.T)
    return np.arccos(np.diag(M))

Now, to illustrate how the random angles depend on the the dimension of the vectors we plot distribution of the sampled angles along with the analytical distribution function.

In [6]:
%matplotlib widget

def plot_angle_distribution(N, samples, vector_function, bins=20):
    sampled_angles = generate_random_angles(N=N, samples=samples,
                                            vector_function=vector_function)
    
    # compute sample distribution
    bin_edges = np.linspace(0, np.pi, bins)
    hist, _ = np.histogram(sampled_angles, bins=bin_edges, density=True)
    
    # clear plot
    plt.cla()
    ax = plt.gca()
    
    # plot sample distribution
    bin_centers = bin_edges[:-1] + np.diff(bin_edges)[0]/2.
    ax.bar(bin_centers, hist, width=.9*np.diff(bin_centers)[0], alpha=0.5, label='sample distribution')
    
    # plot analytic distribution
    phi = np.linspace(0, np.pi, 100)
    ax.plot(phi, [angle_dist(p, N) for p in phi], 'k--', label=r'$\sim\sin^{N-2}\phi$')

    # format figure
    ax.set_xlabel('angle $\phi$')
    ax.set_ylabel('density')
    ax.set_xticks(np.array([0, .25, .5, .75, 1]) * np.pi)
    ax.set_xticklabels(['0', r'$\frac{1}{4}\pi$', r'$\frac{1}{2}\pi$',
                        r'$\frac{3}{4}\pi$', r'$\pi$'])
    ax.set_title('{}D angle distribution'.format(int(N)))
    plt.legend()
    return None

# interactive plot with variable dimension N, and number of sample angles
fig, ax = plt.subplots(figsize=(8,5))
interact(plot_angle_distribution, 
         N=IntSlider(value=2, min=2, max=30, step=1),
         samples=IntSlider(value=10**3, min=0, max=10**4, step=10**3),
         vector_function=fixed(generate_random_vectors), bins=fixed(20));

FigureCanvasNbAgg()

interactive(children=(IntSlider(value=2, description='N', max=30, min=2), IntSlider(value=1000, description='s…

<a id='angles-c'></a>
## Angle distribution for random eigenvectors

But wait! Originally we were interested in the angles between eigenvectors, which we know are not uniformly distributed random vectors because they need to be pairwise orthogonal by definition.
However, this turns out to be only a problem for small dimensions, while the analytical function still very well describes the distribution of these 'eigenangles' in higher dimensions. Let's come back to this and first generate some random eigenvectors.

In order to generate random eigenvectors we need to generate random matricies. Here, we want to focus on correlation matrices, the theory however is easily generalizable to other types of matrices.
Correlation matrices are defined by the following properties.
* The matrix is symmetric.
* All elements are real and $\in$ [$-1,1$].
* The diagonal elements are $1$.
* The matrix is positive definte, i.e., all eigenvalues are $\ge 0$.

A random correlation matrix can be created by calculating the Gram matrix from a set of normalized random vectors $\mathbf{R} = \mathbf{T}\mathbf{T}^*$, where $\mathbf{T}$ is a matrix with rows $T_i$ being noramlized random vectors [(*Homes, 1991*)](). The dimension of the row vectors $T_i$ does not influence the distribution of the eigenvectors. Therefore, we describe this degree of freedom as $\alpha\cdot N$ and arbitrarily choose $\alpha=1$. This has no relevance for the eigenvectors or angle distribution, but will become important later for the eigenvalues.

In [7]:
def generate_random_eigendecomposition(N, samples, alpha):
    nbr_of_matrices = int(samples//N)
    eigenvector_array = np.empty((nbr_of_matrices*N, N))
    eigenvalue_array = np.empty(nbr_of_matrices*N)
    for i in range(nbr_of_matrices):
        random_vectors = np.random.normal(size=(N, alpha*N))
        for j, v in enumerate(random_vectors):
            random_vectors[j] /= np.linalg.norm(v)
        random_corr_matrix = np.dot(random_vectors, random_vectors.T)
        eigenvalues, eigenvectors = sc.linalg.eigh(random_corr_matrix)
        eigenvector_array[i*N:i*N+N] = eigenvectors.T
        eigenvalue_array[i*N:i*N+N] = eigenvalues        
    return eigenvalue_array, eigenvector_array

def generate_random_eigenvectors(N, samples):
    return generate_random_eigendecomposition(N, samples, alpha=1)[1]

The following plot demonstrates how also the angles between eigenvectors reasonably follow the analytic angle distribution when the dimension $N > 10$. 

In [8]:
fig, ax = plt.subplots(figsize=(8,5))
interact(plot_angle_distribution, 
         N=IntSlider(value=2, min=2, max=30, step=1),
         samples=IntSlider(value=10**3, min=0, max=10**4, step=10**3),
         vector_function={'random eigenvectors':generate_random_eigenvectors,
                          'random vectors':generate_random_vectors}, 
         bins=fixed(20));

FigureCanvasNbAgg()

interactive(children=(IntSlider(value=2, description='N', max=30, min=2), IntSlider(value=1000, description='s…

<a id='smallness'></a>
# Angles as similarity measure

<a id='smallness-a'></a>
## Angle smallness

In the beginning, we set out to identify matrix similarity by small angles between their eignvectors. Now that we have a formal description on how to evaluate angle size depending on the vectors' dimension, we can define a corresponding similarity measures, i.e., the *angle-smallness*. So the quantitiy we are actually interested in, is not the absolute angle size but its quantile position in the distribution of random angles. Thus, we define the auxiliary variable $\Delta = 1 - \frac{\phi}{\pi/2}$ to represent the smallness of the angle on a scale form $-1$ to $1$. By performing a simple variable transformation we see that in order to adapt the angle distribution accordingly, we just have to replace the sine function by a cosine and scale with $\frac{\pi}{2}$. 
$$
\tilde{f_\sphericalangle}(\Delta) \propto \cos^{N-2}(\Delta\cdot\pi/2) \qquad \Delta \in [-1, 1] 
$$

In [9]:
def angle_smallness_dist(D, N):
    if D >= -1 and D <= 1:
        return math.gamma(N/2.) / (np.sqrt(np.pi) * math.gamma((N-1)/2)) \
             * np.pi/2 * np.cos(D*np.pi/2)**(N-2)
    else:
        return 0

Another property of eigenvectors, besides being pairwise orthogonal, is that they have an inherent order. They are sorted according to the magnitude of their corresponding eigenvalue. So, the eigenvector with the largest eigenvalue is considered the *first* and argueably the most important, as it represents the axis of largest variance. Consequently, this also means that the angle between the first eigenvectors is a more relevant quantification of the matrices similarity than the angle between the last eigenvectors.

<a id='smallness-b'></a>
## Distribution of eigenvalues

To incorporate this aspect into the test, we want to weight the smallness of each angle with the corresponding eigenvalues. This requires that we know how the eigenvalues are distributed, under the null hypothesis, i.e, for a random matrix.
Fortunately again, the eigenvalue distribution is known for certain types of random matrices, at least in the limit of infinitly sized matrices. For the case of matrices (which includes correlation matrices) of type $\mathbf{Y}_{N}=\mathbf{XX}^{T}$ where $\mathbf{X}$ is a $(\alpha N)\times N$ random matrix whose entries are independent identically distributed random variables with mean $0$ and variance $\sigma^{2}<\infty$, the asymptotic eigenvalue distribution was described by [Vladimir Marcenko and Leonid Pastur in 1967](https://doi.org/10.1070/SM1967v001n04ABEH001994).
This Marchenko-Pastur distribution is only depended on the parameter $\alpha$, which we already introduced as the ratio between the length of the row vectors $T_i$ and the dimensionality $N$. Naturally, the distribution can't depend on $N$ as it assumes an $N \rightarrow \infty$. It is defined as $h_{\alpha}(\lambda)$,

$$
h_{\alpha}(\lambda) = \frac{\alpha}{2 \pi \lambda} \sqrt{(\lambda_+ - \lambda) \cdot (\lambda - \lambda_-)}
\\
\lambda_{\pm} = \left(1 \pm \sqrt{\frac{1}{\alpha}}\right)^2 ,
$$

or as written in Python:

In [10]:
def marchenko_pastur(x, alpha):
    assert alpha >= 1
    x_min = (1 - np.sqrt(1. / alpha)) ** 2
    x_max = (1 + np.sqrt(1. / alpha)) ** 2  
    y = alpha / (2 * np.pi * x) * np.sqrt((x_max - x) * (x - x_min))
    if np.isnan(y):
        return 0
    else:
        return y

To visualize the distribution and its fit to sampled eigenvalues, we again generate random matricies with varying $N$ and $\alpha$. 
And as we can easily observe, the dimension $N$ indeed does not influence the shape eigenvalue distribution. The only effect of increasing $N$ is that the sample variance decreases so that the histogram fits the analytic distribution more closely.

In [11]:
%matplotlib widget

def plot_eigenvalue_distribution(N, alpha, samples, bins=20):
    sampled_eigenvalues, _ = generate_random_eigendecomposition(N=N, 
                                                                alpha=alpha, 
                                                                samples=samples) 
    
    # compute sample distribution
    hist, edges = np.histogram(sampled_eigenvalues, bins=bins, density=True)
    
    # clear plot
    plt.cla()
    ax = plt.gca()

    # plot sample distribution
    bin_centers = edges[:-1] + np.diff(edges)[0]/2
    ax.bar(bin_centers, hist, width=.9*np.diff(bin_centers)[0], alpha=0.5,
           label='sample distribution')
    
    # plot analytic distribution
    xvalues = np.linspace(edges[0], edges[-1], 100)
    ax.plot(xvalues, [marchenko_pastur(x, alpha) for x in xvalues], 'k--',
            label='Marchenko-Pastur distribution')
    
    # format plot
    ax.set_xlabel('eigenvalue $\lambda$')
    ax.set_ylabel('density')
    ax.set_title(r'Eigenvalue distribution ($\alpha = $ {})'.format(alpha))
    plt.legend()
    return ax

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,5))
interact(plot_eigenvalue_distribution, 
         alpha=IntSlider(value=10, min=1, max=500, step=1),
         N=IntSlider(value=10, min=2, max=200, step=1),
         samples=IntSlider(value=1000, min=0, max=2000, step=100),
         bins=fixed(20));

FigureCanvasNbAgg()

interactive(children=(IntSlider(value=10, description='N', max=200, min=2), IntSlider(value=10, description='a…

<a id='smallness-c'></a>
## Weighted angle-smallness

As every angle involves two eigenvectors, we have to combine two eigenvalues for a corrsponding weight. Thus, for the two matrices $\mathbf{A}$ and $\mathbf{B}$ with the eigenvectors $\mathbf{v}_{A,i}$ and $\mathbf{v}_{B,i}$ and eigenvalues $\lambda_{A,i}$ and $\lambda_{B,i}$, we define the weight for the angle-smallness $\Delta$ as the root mean square of the eigenvalues: $w_i \propto \sqrt{(\lambda_{A,i}^2 + \lambda_{B,i}^2)/2}$ with $\sum w_i = N$. By choosing the weights like this, we ensure that distribution of the weights also follows the Marchenko-Pastur distribution $h_{\alpha}$. So, under the null hypothesis of two random matrices we can formulate the distribution of both $\Delta\phi_i$ and the weights $w_i$. To get the distribution of $w_i \cdot \Delta_i$, we combine the two distribution with the following expression $g_{N,\alpha}$.

$$
g_{N,\alpha}(w\Delta) = \int_{\lambda_-}^{\lambda_+} \tilde{f_N}(\frac{\Delta}{\lambda}) \cdot h_{\alpha}(\lambda) \cdot \frac{d\lambda}{\lambda}
$$

In [12]:
def weighted_smallness_dist(D, N, alpha): 
    x_min = (1 - np.sqrt(1. / alpha)) ** 2
    x_max = (1 + np.sqrt(1. / alpha)) ** 2 
    
    integrand = lambda x, _D, _N, _alpha: angle_smallness_dist(_D / float(x), _N) \
                                        * marchenko_pastur(x, _alpha) \
                                        * 1. / x
    return sc.integrate.quad(integrand, x_min, x_max, args=(D,N,alpha,))[0]

Let's check out how this combined distribution behaves in action.

In [13]:
def plot_weighted_smallness_distribution(N, alpha, samples, bins=20):
    # generate pairs of random eigenvectors and values
    eigenvalues_A, eigenvectors_A = generate_random_eigendecomposition(N=N, 
                                                                       samples=samples, 
                                                                       alpha=alpha)

    eigenvalues_B, eigenvectors_B = generate_random_eigendecomposition(N=N, 
                                                                       samples=samples, 
                                                                       alpha=alpha)
    # calculate eigenangles and their smallness
    M = np.dot(eigenvectors_A, eigenvectors_B.T)
    angles = np.arccos(np.diag(M))
    smallness = 1 - angles / (np.pi/2.)

    # calculate weights and apply to smallness
    weights = np.sqrt((eigenvalues_A ** 2 + eigenvalues_B ** 2) / 2.)
    weights = weights / sum(weights) * N
    weighted_smallness = smallness * weights 
    weighted_smallness = weighted_smallness * len(smallness)/N # correct for multiple matrices
    
    # compute sample distribution
    bin_edges = np.linspace(-1, 1, bins)
    hist, _ = np.histogram(weighted_smallness, bins=bin_edges, density=True)
    
    # clear plot
    plt.cla()
    ax = plt.gca()
    
    # plot sample distribution
    bin_centers = bin_edges[:-1] + np.diff(bin_edges)[0]/2.
    ax.bar(bin_centers, hist, width=.9*np.diff(bin_centers)[0], alpha=0.5, 
           label='sample distribution')
    
    # plot the analytical distribution
    xvalues = np.linspace(-1, 1, 100)
    ax.plot(xvalues, [weighted_smallness_dist(x, N, alpha) for x in xvalues], 'k--',
            label=r'analytical distribution $g_{N,\alpha}$')
    
    # format plot
    ax.set_xlabel(r'weighted angle-smallness $w\cdot\Delta$')
    ax.set_ylabel('density')
    ax.set_title(r'weighted angle-smallness ({}D, $\alpha=${})'.format(N, alpha))
    plt.legend()
    return None
    
fig, ax = plt.subplots()
interact(plot_weighted_smallness_distribution, 
         alpha=IntSlider(value=10, min=1, max=500, step=1),
         N=IntSlider(value=10, min=2, max=200, step=1),
         samples=IntSlider(value=1000, min=0, max=2000, step=100),
         bins=fixed(20));

FigureCanvasNbAgg()

interactive(children=(IntSlider(value=10, description='N', max=200, min=2), IntSlider(value=10, description='a…

<a id='test'></a>
# Building the statistical test

<a id='test-a'></a>
## Similarity score

We use the quantity of the weighted angle-smallness as a measure for the matrix similarity, but to evaluate the similarity of two $N\times N$ matrices it is not very handy to have $N$ different values. Thus, we average over the $N$ to arrive at a single valued similariy score.
$$
\eta = \frac{1}{N} \sum_i^N w_i \cdot \Delta_i
$$
A large $\eta$ indicates that on average the weighted angles between the eigenvectors are smaller than would be expected for random matrices, and this we initialy wanted to measure as similarity. 

In order to interpret a given $\eta$ for a sample with a certain $N$ and $\alpha$, we also need to know how $\eta$ is theoretically distributed. Since we know how $w \Delta \phi$ is distributed and $\eta$ is the average of $N$ samples of $w \Delta \phi$, for a sufficiently large $N$ we can apply the central limit theorm.

The central limit theorem states that the mean of a set of idependent sample values (here $w_i\Delta_i$) tends to be normally distributed, regardless of the distribution shape for the sample values. This normal distribution centers around the expected mean (here $0$) with the standard deviation $\sigma = \frac{s}{\sqrt{N}}$, where $s$ is the standard deviation of $g_{N,\alpha}(w\Delta)$. Following the definition for the standard deviation for continous distributions, we can calculate $s$ as the integral over the product of the distribution with the squared distance to the mean. 

So, finally we arrive at an analytical description of the similarity score distribution $f(\eta)$.

$$
f(\eta) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp(-\frac{\eta^2}{2\sigma^2})\\
\sigma^2=  \frac{1}{N} \int x^2 \cdot g_{N,\alpha}(x) \ dx
$$

In [14]:
def similarity_score_distribution(eta, N, alpha):
    integrand = lambda x, N_, alpha_: x**2 * weighted_smallness_dist(x, N_, alpha_)
    var = sc.integrate.quad(integrand, -np.infty, np.infty, args=(N,alpha,))[0]
    sigma = np.sqrt(var/N)
    return sc.stats.norm.pdf(eta, 0, sigma)

For a final check we plot a sample histogram of similarity scores alongside the theoretical distribution. In contrast to the previous plots, the number of samples in the histogram is no longer the number of vector pairs or eigenvalues, but the number of matrix pairs. Thus, to avoid confusion we call them *runs*. 

Because, each matrix pair generates only one similarity score, contrasting to $N$ eigenangles, it takes considerably longer to sample a reasonable amout to visualize them.

In [15]:
def sample_similarity_scores(N, alpha, runs):
    similarity_scores = np.zeros(runs)
    
    samples = N

    for run in range(runs): 
        # generate pairs of random eigenvectors and values
        eigenvalues_A, eigenvectors_A = generate_random_eigendecomposition(N=N, 
                                                                           samples=samples, 
                                                                           alpha=alpha)

        eigenvalues_B, eigenvectors_B = generate_random_eigendecomposition(N=N, 
                                                                           samples=samples, 
                                                                           alpha=alpha)
        M = np.dot(eigenvectors_A, eigenvectors_B.T)
        angles = np.arccos(np.diag(M))
        smallness = 1 - angles / (np.pi/2.)

        # calculate weights and apply to smallness
        weights = np.sqrt((eigenvalues_A ** 2 + eigenvalues_B ** 2) / 2.)
        weights = weights / sum(weights) * N
        weighted_smallness = smallness * weights

        similarity_scores[run] = np.mean(weighted_smallness)
    
    return similarity_scores

In [16]:
N = 10
alpha = 500
runs = 1000

similarity_scores = sample_similarity_scores(N, alpha, runs)

In [17]:
fig, ax = plt.subplots()
bins = 20
xlims = (-.25, .25)

# compute sample distribution
bin_edges = np.linspace(xlims[0], xlims[1], bins)
hist, _ = np.histogram(similarity_scores, bins=bin_edges, density=True)

# plot sample distribution
bin_centers = bin_edges[:-1] + np.diff(bin_edges)[0]/2.
ax.bar(bin_centers, hist, width=.9*np.diff(bin_centers)[0], alpha=0.5, 
       label='sample distribution')

# plot the analytical distribution
xvalues = np.linspace(xlims[0], xlims[1], 100)
ax.plot(xvalues, similarity_score_distribution(xvalues, N, alpha), 'k--',
        label=r'analytical distribution $f(\eta)$')

# format plot
ax.set_xlabel(r'similarity scores $\eta$')
ax.set_ylabel('density')
ax.set_title(r'Null distribution of eigenangle test ({}D, $\alpha=${})'.format(N, alpha))
plt.legend()
plt.show()

FigureCanvasNbAgg()

<a id='test-b'></a>
## Null hypothesis
The basis of the null hypothesis is that the two matrices are independent, so that the angles between their eigenvectors can be described by a random angle distribution.

Furthermore we assumed that $N$ is large, so that the constraint that eigenvectors are orthogonal could be neglected for the angle distribution, and so that we could apply the central limit theorem in the last step. We assumed the eigenvalues follow a Marchenko-Pastur distribution, which requires as well an $N \rightarrow \infty$ and the matrices to be of the type $\mathbf{Y}_{N}=\mathbf{XX}^{T}$, as described in the section [*Distribution of eigenangles*](#smallness-b). With the numerical simulations along the way we found that $N>10$ is already sufficient so that by visual inspection the null distribution $f(\eta)$ reasonably represents the randomly sampled test data.

The working principle of a Null Hypothesis Significance Test (NHST) is that any violations of the null hypothesis in real data results in a score which lies more in the margins of the distribution, and is therefore less likely to be explained by stochastic variation. However, there are many different ways how the null hypothesis could be violated. For example, $N$ could be small, the matrices or subsets of the matrices could be correlated or anti-correlated, or the matrices slightly deviate from being symmetric. There is no reason to assume that the test is equally susceptive to the  different kinds of deviations from the null hypothesis. Without extensive calibration it therefore is very hard to say exactly how well the test detects similarity, and what this similarity means in detail.

<a id='test-c'></a>
## P-value and interpretation

As long as the null hypothesis is fullfiled for a dataset, the resulting similarity score will be a random variable with the probability distribution $f(\eta)$. Here, the distribution is centered around $0$, so $0$ would be the most likely value to expect. The idea is that when the null hypothesis is not applicable anymore and the matrices are correlated, the similarity score will be larger. Ideally, a substantial correlation between the matrices results in a score which is very unlikely to be explained just by the stochastic spread of the null distribution.
To quantify this evaluation, the probability of a score-value to be a random variable of the null distribution is defined as the integral over the quantile of distribution the value falls into.
$$
p_{\eta} = 2 \int_{|\eta|}^{\infty} f(x) \ dx
$$
This probability value is commonly refered to as the $p$-value* of the statistical test, or more precisely the two-sided $p$ value.
However, here we are only interested in scores which deviate two larger values, i.e. the right hand side of the distribution. Thus, our test is a one-sided test. Deviation to large negative values are also unlikely to be explained by the null distribution, but do not indicate similarity. On the contrary, in that scenario the corrsponding eigenvectors of the two matrices tend to be perpendicular, which could be interpreted as an anti-correlation. So in order to think of the $p$ value as the probability of non-similarity, we use the one-sided $p$ value
$$
p_{\eta} = \int_{\eta}^{\infty} f(x) \ dx.
$$
This means as well that the average score $\eta=0$ corresponds to a $p$ value of $0.5$, and scores with large negative values have $p$ values $\rightarrow 1$. So as with every NHST we do not test the property of interest (here similarity) directly, but conclude it as an alternative hypothesis when the null hypothesis (here independence) is rejected due to a small $p$ value.

Note: Historically, the decision whether or not to reject the null hypothesis is often done by setting a rather arbitrary significance level (e.g. $0.05$) and considering the $p$ value 'significant' when it's smaller and therefore reject the null hypothesis. There are various problems with that which are discussed in detail in various articles and papers, for example [here](https://www.nature.com/articles/d41586-019-00857-9), [here](https://doi.org/10.3389/fnhum.2017.00390), and [here](https://doi.org/10.1111/j.1469-185X.2007.00027.x). Generally, a good practice to work with a $p$ value is to regard it as another quantification, which is a random variable and has a distribution on its own. Thus, to make proper sense of it, its best combined with additional measures such as, for example, its confidence interval, an effect size, and an analysis of statistical power.

In [22]:
N = 20
alpha = 1000

# calculate distribution spread to adjust plotting limits
integrand = lambda x, N_, alpha_: x**2 * weighted_smallness_dist(x, N_, alpha_)
var = sc.integrate.quad(integrand, -np.infty, np.infty, args=(N,alpha,))[0]
sigma = np.sqrt(var/N)

score = sample_similarity_scores(N, alpha, runs=1)

# integrate over null distribution from -inf to score
pvalue = sc.integrate.quad(similarity_score_distribution, score, np.inf, args=(N, alpha))[0]


fig, ax = plt.subplots()

# plot the analytical distribution
xvalues = np.linspace(-5*sigma, 5*sigma, 100)
ax.plot(xvalues, similarity_score_distribution(xvalues, N, alpha), 'k--',
        label=r'null distribution $f(\eta)$')

# plot score and area ~ pvalue
ax.axvline(score, color='r', label=r'sample score $\eta$')
right_to_score = np.linspace(score, 5*sigma, 100)
ax.fill_between(right_to_score, 
                similarity_score_distribution(right_to_score, N, alpha),
                alpha=0.6, label='~ p-value')
ax.text(score+var/4, .2/sigma, 'p-value = {:.2}'.format(pvalue))

# format plot
ax.set_xlabel(r'similarity $\eta$')
ax.set_ylabel('probability density')
ax.set_ylim((0,0.45/sigma))
plt.legend();

FigureCanvasNbAgg()

<a id='apply'></a>
# Application to neural activity data

<a id='apply-a'></a>
## Practical considerations

<a id='apply-b'></a>
## Example data

<a id='apply-c'></a>
## Parameter scan